# Mlab Sites analysis

In [4]:
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geohash
import cartopy.crs as ccrs
from datetime import datetime
import geohash
import geopandas
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
from matplotlib import colors
from descartes import PolygonPatch
from unidecode import unidecode
from matplotlib.patches import Patch

# Setting global font to "CMU Sans Serif"
from matplotlib import rcParams
rcParams["font.family"] = "CMU Sans Serif"

# This will compile plots to be Latex compatible
plt.rc('text', usetex=True)

In [ ]:
def plot_probe_map(city_overview_df, city_df, interesting_cities,
                       lon_bounds = (-180, 180),
                       lat_bounds = (-90, 90),
                       figsize=(7, 6),
                       annotate = True,
                       annotate_minimal = False,
                       savefig = "",
                       figures_path="figures"):
    city_colors = {city: cols[i % len(cols)] for i, city in enumerate(interesting_cities)}
    lstyles = {city: lstyle_array[i % len(lstyle_array)] for i, city in enumerate(interesting_cities)}
    
    client_country_coords_df = city_overview_df.groupby(["ClientCountry", "ClientCity"])[["lat", "lon"]].mean()
    client_country_coords_df = client_country_coords_df[(
                                    client_country_coords_df["lat"] > lat_bounds[0]) & (client_country_coords_df["lat"] < lat_bounds[1]
                                                        )]
    client_country_coords_df = client_country_coords_df[(
                                    client_country_coords_df["lon"] > lon_bounds[0]) & (client_country_coords_df["lon"] < lon_bounds[1]
                                                        )]

    world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
    world = world[(world.name!="Antarctica")]

    def plotCountryPatch( axes, country_name, fcolor ):
        # plot a country on the provided axes
        nami = world[world.name == country_name]
        namigm = nami.__geo_interface__['features']  # geopandas's geo_interface
        namig0 = {'type': namigm[0]['geometry']['type'], \
                  'coordinates': namigm[0]['geometry']['coordinates']}
        axes.add_patch(PolygonPatch( namig0, fc=fcolor, ec="black", alpha=0.2, zorder=2, linewidth=0.0 ))


    fig, ax = plt.subplots(figsize=figsize, subplot_kw={'projection': ccrs.PlateCarree()})

    ax.set_xlim(lon_bounds)
    ax.set_ylim(lat_bounds)

    cmap = colors.ListedColormap(cols[:-2])
    world.plot(facecolor = "grey", edgecolor="black", ax=ax, linewidth=0.2, alpha=0.2, zorder=0)

    for c in countries:
        c = "United States of America" if c == "United States" else c
        c = "Czechia" if c == "Czech Republic" else c

        if(c in world["name"].values):
            plotCountryPatch(ax, c, "#364aa088")

    client_country_df = city_overview_df.groupby(["ClientCountry", "ClientCity"])[["lat", "lon"]].mean()
    latitude, longitude = client_country_coords_df["lat"], client_country_coords_df["lon"]
    ax.scatter(longitude, latitude, 
                   sizes = [0.7],
                   color="#a8290a88",
                   zorder = 10)

    # Filtered with interesting_cities
    client_country_df = city_overview_df[city_overview_df["ClientCity"].isin(interesting_cities)].groupby(["ClientCountry", "ClientCity"])[["lat", "lon"]].mean()
    latitude, longitude = client_country_df["lat"], client_country_df["lon"]
    ax.scatter(longitude, latitude, 
                   sizes = [10.1],
                   color="green",
                   zorder = 9, marker="x")

    for x, y, label in zip(client_country_coords_df["lon"], client_country_coords_df["lat"], client_country_coords_df.index):
        if(annotate and ((not annotate_minimal) or (label[1] in interesting_cities))):
            ax.annotate(label[1], xy=(x, y), xytext=(-7, 5), textcoords="offset points")

    plt.tight_layout()
    plt.show()

In [2]:
df = pd.read_csv("./data/mlab_sites.csv")

In [3]:
df

,Site,CountryCode,City,Latitude,Longitude,ASNumber
0,NaN,NaN,NaN,NaN,NaN,NaN
1,akl01,NZ,Auckland,-36.8500,174.7830,38022.0
2,ams03,NL,Amsterdam,52.3086,4.7639,1299.0
3,ams04,NL,Amsterdam,52.3086,4.7639,3257.0
4,ams05,NL,Amsterdam,52.3086,4.7639,1273.0
...,...,...,...,...,...,...
252,yyz04,CA,Toronto,43.6767,-79.6306,6453.0
253,yyz05,CA,Toronto,43.6767,-79.6306,1299.0
254,yyz06,CA,Toronto,43.6767,-79.6306,174.0
255,yyz07,CA,Toronto,43.6767,-79.6306,396982.0
